In [1]:
# -*- coding: utf-8 -*
# author ChangeBio

from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torchvision
from torch.optim import Adam
from torch.autograd import Variable

#from flyai.dataset import Dataset
from flyai.source.base import DATA_PATH

from model import Model
from net import Net, LSTMNet
from path import MODEL_PATH
from processor import DQuestionFlyAI,load_data, make_weights_for_balanced_classes
from utils import Bunch

def eval(model, x_test, y_test):
    network.eval()
    total_acc = 0.0
    data_len = len(x_test[0])
    x1, x2 = x_test
    x1 = torch.from_numpy(x1)
    x2 = torch.from_numpy(x2)
    x1 = x1.float().to(device)
    x2 = x2.float().to(device)
    y_test = torch.from_numpy(y_test)
    y_test = y_test.to(device)
    batch_eval = model.batch_iter(x1, x2, y_test)

    for x_batch1, x_batch2, y_batch in batch_eval:
        outputs = network(x_batch1, x_batch2)
        _, prediction = torch.max(outputs.data, 1)
        correct = (prediction == y_batch).sum().item()
        total_acc += correct
    return total_acc / data_len

parser = argparse.ArgumentParser()
parser.add_argument("-e", "--EPOCHS", default=10, type=int, help="train epochs")
parser.add_argument("-b", "--BATCH", default=16, type=int, help="batch size")
args = parser.parse_args([])

#settings
settings = {
'net':'densenet121',
'nc':2,    
'lr': 0.001,
'seed': 1,
'log_interval': 100,
'save_model': True}
print("1.settings",settings)
settings = Bunch(settings)
use_cuda = torch.cuda.is_available()
torch.manual_seed(settings.seed)
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {'num_workers': 0, 'pin_memory': True} if use_cuda else {}

#load data
#data = Dataset()
#model = Model(data)
trn,val = load_data()
train_dataset = DQuestionFlyAI(root=DATA_PATH,df=trn)
val_dataset = DQuestionFlyAI(root=DATA_PATH,df=val)
weight = make_weights_for_balanced_classes(trn.label,settings.nc)
sampler = torch.utils.data.sampler.WeightedRandomSampler(weight,len(weight))
train_loader = torch.utils.data.DataLoader(train_dataset,
    batch_size=args.BATCH, sampler =sampler, **kwargs)
test_loader = torch.utils.data.DataLoader(val_dataset,
    batch_size=args.BATCH, **kwargs)
n_train = len(train_dataset)
batch_train = n_train/args.BATCH
print(args.BATCH,n_train/args.BATCH)
n_test = len(val_dataset)
batch_test = n_test/args.BATCH
print("2. load data. train_dataset %d,batch %d, val_dataset %d, batch %d." % (n_train,batch_train,n_test,batch_test))

1.settings {'net': 'densenet121', 'nc': 2, 'lr': 0.001, 'seed': 1, 'log_interval': 100, 'save_model': True}
{'id': 'ReadingComprehesionCh', 'type': 'csv', 'config': {'train_url': 'https://dataset.flyai.com/dataset/ReadingComprehesionCh/dev.zip', 'test_url': 'https://dataset.flyai.com/dataset/ReadingComprehesionCh/dev.zip'}}
[200.0, 0.03, 0.1710152950930964, 0.0, 0.0, 0.0, 0.0, 1.0]
Barnes & Noble                                                                                                                                         2
营养素含量                                                                                                                                                  2
地处山麓台地与沿海平原交接地带，为瀑布线城市之一。                                                                                                                              2
吻部坚硬锋利。                                                                                                                                                2
我想我

In [3]:
x1,x2, y_train = next(train_iter)

In [6]:
temp = [train_dataset[i] for i in range(5)]

In [7]:
type(temp)

list

In [8]:
temp[0]

(array([[ 0.464023, -0.055275, -0.358562, ..., -0.638851, -0.109829,
          0.397766],
        [ 0.325058, -0.380674,  0.468678, ..., -0.647425, -0.230343,
          0.366162],
        [ 0.333389,  0.254132,  0.381571, ..., -0.035394, -0.017113,
          0.322221],
        ...,
        [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
          0.      ],
        [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
          0.      ],
        [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
          0.      ]]),
 array([[ 0.223334, -0.032572,  0.12772 , ..., -0.134228, -0.200968,
         -0.089978],
        [-0.134102, -0.393164, -0.030239, ..., -0.221924, -0.166869,
         -0.088888],
        [ 0.209092, -0.165459, -0.058054, ..., -0.278624, -0.173408,
          0.035439],
        ...,
        [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
          0.      ],
        [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
   

In [2]:
#load net structure
print("3.load net structure: %s, number of class: %d" % (settings.net,settings.nc))
net = LSTMNet()
gpu = torch.cuda.is_available()
if gpu:
    net.cuda()
#optimize and loss
print("4.optimize and loss. learning rate %g" % settings.lr)
optimizer = Adam(net.parameters(), lr=settings.lr, weight_decay=1e-4)
#optimizer = Adam(net.parameters(), lr=0.001, betas=(0.9, 0.999))  
loss_fn = nn.CrossEntropyLoss()  

#train and test
print("5.***************train and test*********************")
best_accuracy = 0
train_iter = iter(train_loader)
batch_idx = 0
for epoch in range(args.EPOCHS):
    net.train()
    #x_train, y_train, x_test, y_test = data.next_batch(args.BATCH)  # 读取数据
    #batch_len = y_train.shape[0]
    #x1, x2 = x_train
    #x1 = torch.from_numpy(x1)
    #x2 = torch.from_numpy(x2)
    #x1 = x1.float().to(device)
    #x2 = x2.float().to(device)
    #y_train = torch.from_numpy(y_train)
    #y_train = y_train.to(device)
    try:
        batch_idx +=1
        x1,x2, y_train = next(train_iter)
        #print(batch_idx,"data len",len(x_train),len(y_train))
    except:
        batch_idx = 0
        train_iter = iter(train_loader)
        x1,x2, y_train = next(train_iter)
        print(len(x1),len(x2),len(y_train),"data len")

    if gpu:
        x1 = Variable(x1.cuda())
        x2 = Variable(x2.cuda())
        y_train = Variable(y_train.long().cuda())


    outputs = net(x1,x2)
    optimizer.zero_grad()
    loss = loss_fn(outputs, y_train)
    loss.backward()
    optimizer.step()
    if batch_idx % settings.log_interval == 0:
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batch_idx * len(y_train), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.item()))

if settings.save_model:       
    model.save_model(net, MODEL_PATH, overwrite=True)
     

3.load net structure: densenet121, number of class: 2


Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache


4.optimize and loss. learning rate 0.001
5.***************train and test*********************


Loading model cost 0.473 seconds.
Prefix dict has been built succesfully.


NameError: name 'model' is not defined